# 프로젝트 B: 딥러닝 기반 영화 리뷰 감성 분석 시스템 (실습 문제)

## 실습 목표
- 딥러닝 기법을 활용한 감성 분석 시스템을 직접 구현
- Hugging Face Pipeline과 ClovaX 두 가지 방법을 비교
- 네이버 영화평 데이터를 이용한 실전 감성 분석 수행

## 참고 자료
- **정답 코드**: `701_Project_Movie_감성분석` (참고용)
- **Hugging Face Pipeline**: `300_HuggingFace_QuickStart_pipeline_tokenizer` 참고
- **ClovaX**: `601_Text_Generation_Prompts_CLOVAX` 참고

---

## 단계 1: 데이터 준비 및 최소 전처리

**목표**: 네이버 영화평 데이터를 다운로드하고 딥러닝 모델에 적합한 최소한의 전처리 수행

**지시사항**:
1. 필요한 라이브러리를 import하세요:
   - `pandas`, `numpy`, `tensorflow`
   - `transformers`에서 `pipeline`, `AutoModelForCausalLM`, `AutoTokenizer` import
   - `huggingface_hub`에서 `login` import
   - `dotenv`에서 `load_dotenv` import
   - `sklearn.metrics`에서 `accuracy_score`, `confusion_matrix`, `classification_report` import
   - `os` import

2. `.env` 파일에서 `HUGGING_FACE_TOKEN`을 로드하고 Hugging Face에 로그인하세요.
   - `load_dotenv()` 사용
   - `os.getenv("HUGGING_FACE_TOKEN")`으로 토큰 가져오기
   - 토큰이 있으면 `login(token=token)` 사용

3. `tensorflow.keras.utils.get_file()`을 사용하여 다음 URL에서 데이터를 다운로드하세요:
   - 훈련 데이터: `https://raw.github.com/ironmanciti/Infran_NLP/master/data/naver_movie/ratings_train.txt`
   - 테스트 데이터: `https://raw.github.com/ironmanciti/Infran_NLP/master/data/naver_movie/ratings_test.txt`

4. `pd.read_csv()`를 사용하여 데이터를 로드하세요 (구분자는 `'\t'`)

5. 결측값을 제거하세요 (`dropna()` 사용)

6. 처리 속도 향상을 위해 데이터를 샘플링하세요:
   - 훈련 데이터: 5,000개 (random_state=1)
   - 테스트 데이터: 1,000개 (random_state=1)

7. `document` 컬럼을 `str` 타입으로 변환하고, 빈 문자열을 가진 행을 제거하여 `cleaned_document` 컬럼을 생성하세요.

**참고**: `701_Project_Movie_감성분석` 섹션 1 참고

In [ ]:
# 여기에 데이터 로드 및 최소 전처리 코드를 작성하세요

---
## 단계 2: 감성 분석 방법 1: Hugging Face Pipeline

**목표**: Hugging Face `pipeline`을 사용하여 BERT 기반 감성 분석 수행 및 평가


### 2.1 파이프라인을 이용한 감성 분석

**지시사항**:
1. `nlptown/bert-base-multilingual-uncased-sentiment` 모델을 사용하여 `sentiment-analysis` 파이프라인을 생성하세요.
   - `pipeline('sentiment-analysis', model=model_name)` 사용

2. 다음 샘플 리뷰에 대해 감성 분석을 수행하고 결과를 출력하세요:
   ```python
   sample_texts = [
       "다시는 보고 싶지 않은 짜증나는 영화",
       "아주 재미있는 영화",
       "정말 재미없는 영화였다",
       "이 영화 최고",
       "보통 영화"
   ]
   ```

3. 각 리뷰에 대해 `label`과 `score`를 출력하세요.

**참고**: `300_HuggingFace_QuickStart_pipeline_tokenizer`의 `pipeline` 사용법 참고


In [ ]:
# 여기에 Hugging Face 샘플 감성 분석 코드를 작성하세요

### 2.2 테스트 데이터 감성 분석

**목표**: Hugging Face 파이프라인으로 테스트 데이터의 감성을 분석하고 정확도 평가

**지시사항**:
1. `df_test`의 `cleaned_document` 컬럼에서 상위 10개 리뷰를 추출하여 `test_samples` 리스트를 만드세요.

2. 해당 리뷰들의 실제 레이블(`df_test['label']`)을 `test_labels` 리스트로 만드세요.

3. `sentiment_classifier`를 사용하여 `test_samples`의 감성을 분석하세요.

4. 각 리뷰에 대해 실제 레이블과 예측 레이블을 비교하고, 정확도를 계산하여 출력하세요.
   - 모델의 별점(1 ~ 5)을 긍정(4 ~ 5점) 또는 부정(1 ~ 2점)으로 변환하여 실제 레이블과 비교하세요.
   - 예: `predicted_label = 1 if int(result['label'].split()[0]) >= 4 else 0`
   - 정확도는 `accuracy_hf` 변수에 저장하세요.

**참고**: `701_Project_Movie_감성분석` 섹션 2.2 참고


In [ ]:
# 여기에 Hugging Face 테스트 데이터 감성 분석 코드를 작성하세요

---
## 단계 3: 감성 분석 방법 2: ClovaX

**목표**: ClovaX 모델을 사용하여 LLM 기반 감성 분석 수행 및 평가


### 3.1 ClovaX 모델 로드 및 감성 분석 함수 작성

**지시사항**:
1. ClovaX 모델을 로드하세요:
   - 모델 이름: `"naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"`
   - `AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")` 사용
   - `AutoTokenizer.from_pretrained(model_name)` 사용

2. `analyze_sentiment_clovax(text)` 함수를 작성하세요.
   - 이 함수는 주어진 텍스트에 대해 ClovaX 모델을 사용하여 감성을 "긍정" 또는 "부정"으로 분류해야 합니다.
   - `system_content`와 `user_content`를 설정하여 프롬프트를 구성하세요.
   - `chat` 형식으로 대화 내용을 구성: `[{"role": "tool_list", "content": ""}, {"role": "system", "content": system_content}, {"role": "user", "content": user_content}]`
   - `clovax_tokenizer.apply_chat_template()`을 사용하여 입력을 변환하세요.
   - `clovax_model.generate()`를 사용하여 응답을 생성하세요.
   - 생성된 텍스트에서 "긍정" 또는 "부정" 키워드를 찾아 1 또는 0으로 반환하세요.

3. Hugging Face와 동일한 샘플 리뷰(`sample_texts`)에 대해 `analyze_sentiment_clovax` 함수를 사용하여 감성 분석을 수행하고 결과를 출력하세요.

**참고**: `601_Text_Generation_Prompts_CLOVAX`의 ClovaX 사용 예시 참고


In [ ]:
# 여기에 ClovaX 모델 로드 및 샘플 감성 분석 코드를 작성하세요

### 3.2 테스트 데이터 감성 분석

**목표**: ClovaX로 테스트 데이터의 감성을 분석하고 정확도 평가

**지시사항**:
1. Hugging Face와 동일한 `test_samples`와 `test_labels`를 사용하세요.

2. 각 리뷰에 대해 `analyze_sentiment_clovax` 함수를 호출하여 예측 레이블을 얻으세요.

3. 실제 레이블과 예측 레이블을 비교하고, 정확도를 계산하여 출력하세요.
   - 정확도는 `accuracy_clovax` 변수에 저장하세요.

**참고**: `701_Project_Movie_감성분석` 섹션 3.2 참고


In [ ]:
# 여기에 ClovaX 테스트 데이터 감성 분석 코드를 작성하세요

---
## 체크리스트

각 단계를 완료한 후 다음을 확인하세요:

- [ ] 단계 1: 데이터가 정상적으로 로드되고 최소 전처리가 수행되었는가?
- [ ] 단계 2: Hugging Face Pipeline으로 샘플 및 테스트 데이터 감성 분석이 올바르게 수행되고 정확도가 출력되는가?
- [ ] 단계 3: ClovaX 모델이 로드되고, 샘플 및 테스트 데이터 감성 분석이 올바르게 수행되고 정확도가 출력되는가?
- [ ] 단계 4: 두 방법의 비교표가 올바르게 생성되고 출력되는가?